In [ ]:
"""
Examples 有提供各案例及 notebook
https://microsoft.github.io/autogen/docs/Examples
https://microsoft.github.io/autogen/
https://microsoft.github.io/autogen/docs/Use-Cases/enhanced_inference/#api-unification
https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat/
https://microsoft.github.io/autogen/docs/llm_configuration
https://github.com/microsoft/autogen/blob/main/notebook/config_loader_utility_functions.ipynb
https://microsoft.github.io/autogen/blog/
"""

In [ ]:
import autogen
from autogen.cache import Cache
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

# Load LLM inference endpoints from an environment variable or a file
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")
"""
20240219，此處列表更新為5類
gpt35、gpt4、gpt4v、
gemini、gemini_vision(gemini 還不大行，會報錯，暫不支援)
cohere 可以調用，但免費的很快超過額度
groq   可以調用，但有點怪怪的。
也加入本地大模型，但相容性似乎都有問題，第一輪對話還可以，第一輪之後都不行。
- ollama：要使用 litellm
- LM Studio 
- Jan (2024/03/02 開啟GPU加速會報錯)
"""
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-16k-0613"],
    },
)
config_list_ollama = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["ollama"],
    },
)
config_list_LM_Satudio = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["LM Studio"],
    },
)
config_list_jan = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["OpenHermes Neural 7B Q4"],
    },
)
config_list_groq_llama2 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["llama2-70b-4096"],
    },
)
config_list_groq_mixtral = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["mixtral-8x7b-32768"],
    },
)
config_list_cohere = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["command-nightly"],
    },
)
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0613", "gpt-4-0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview", "dalle"],
    },
)
config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)
config_list_gemini_vision = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro-vision"],
    },
)


In [ ]:
# 這裡設定要用那一個大模型，試吧，GPT 的幾乎沒遇過問題，不是 GPT 的，永遠都是在遇到問題
# config_list = config_list_groq_llama2
config_list = config_list_ollama

In [ ]:
assistant = AssistantAgent("assistant", llm_config={"config_list": config_list},max_consecutive_auto_reply=4)
user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False}, llm_config={"config_list": config_list}, 
                            human_input_mode="NEVER",max_consecutive_auto_reply=4
                            )
"""
# 這裡使用 Chche 的目的?
with Cache.disk() as cache:
    # start the conversation
    user_proxy.initiate_chat(
        assistant,
        message="Draw two agents chatting with each other with an example dialog.",
        # message="Draw two agents chatting with each other with an example dialog. Don't add plt.show().",
        cache=cache,
    )
"""
# Kickstart a conversation between the agents to plot a stock price chart
user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")


In [ ]:
import autogen
from IPython import get_ipython
from typing_extensions import Annotated

# 限定只用 gpt3.5
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["ollama"],
    },
)

In [ ]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# user_proxy 負責執行程式碼 或是 獲取人類的輸入
user_proxy=autogen.UserProxyAgent(
        name="user_proxy",
        human_input_mode="NEVER",
    	code_execution_config={"work_dir": "coding", "use_docker": False},
        is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
        )
# 
assistant=autogen.AssistantAgent(
        name="assistant",
	system_message="You are a helpful assistant. Reply TERMINATE when the task is done.",
        llm_config={"config_list":config_list, 
                    # "seed": 42,
                    }, 
	max_consecutive_auto_reply=4,
)

In [ ]:
# 設定任務 task
task = "Solve x^2 + 3x + 2 = 0, and explained the solution in Traditional Chinese."

In [ ]:
# 開始對話解題
user_proxy.initiate_chat(assistant,message=task)

In [ ]:
# 用"function call"來實現向專家提問的拓展。但下一個區塊會報錯"Error: Function ask_expert not found."
# 修正方式如下：
"""
# 會用到 Annotated，先匯入
from typing_extensions import Annotated
# 註冊函數跟定義函式的格式
@user_proxy.register_for_execution()
@assistant.register_for_llm(name="expert", description="expert to solve math question.")
def expert(message: Annotated[str, "Math question to solve."]) ->str:
"""
@user_proxy.register_for_execution()
@assistant.register_for_llm(name="expert", description="expert to solve math question.")
def expert(message: Annotated[str, "Math question to solve."]) ->str:
	assistant_for_expert = autogen.AssistantAgent(
		name = "assistant_for_expert",
		llm_config = {
			"temperature":1,
			"config_list":config_list
			}
		)
	expert = autogen.UserProxyAgent(
		name = "expert",
		human_input_mode = "NEVER",  # ALWAYS會一直要求輸入，不知道要輸入什麼，不然就改成 NEVER
		code_execution_config = {"work_dir":"expert", "use_docker": False}
		)
	expert.initiate_chat(assistant_for_expert, message = message)
	expert.stop_reply_at_receive(assistant_for_expert)
	expert.send("解出答案並以淺顯易懂的方式解釋怎麼解題", assistant_for_expert)
	return expert.last_message()["content"]

In [ ]:
# define functions according to the function description
# 應該不需要 exec_python 這個 tools? user_proxy 不是已經有 code_execution_config={"work_dir": "coding", "use_docker": False}, ？？
# 第一種方法:使用 register_for_execution 及 register_for_llm 二個裝飾器來註冊函數

@user_proxy.register_for_execution()
@assistant.register_for_llm(name="python", description="run cell in ipython and return the execution result.")
def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log

# 第二種方法:使用 register_function 來註冊函數
def exec_sh(script: Annotated[str, "Valid Python cell to execute."]) -> str:
    return user_proxy.execute_code_blocks([("sh", script)])

autogen.agentchat.register_function(
    exec_python,
    caller=assistant,
    executor=user_proxy,
    name="sh",
    description="run a shell script and return the execution result.",
)


In [ ]:
# 設定任務 task
task = "Solve x^2 + 3x + 2 = 0, and explained the solution in Traditional Chinese."

In [ ]:
# 開始對話解題
user_proxy.initiate_chat(assistant,message=task)
